# III. Transfer Learning : VGG-16 & Algorithm Creation

In this notebook, we shall try to create a model using top notch Convolutional Neural Networks for image Classification : 
  
  * VGG16 

On the previous jupyter file (II. Transfer learning : DENSENET 161) we obtained an accuracy of 82%, way higher than on the first model created from sctatch (10%) and with only a very few epochs.

We'll see what happens to accuracy when testing VGG-16

Since this model produces the best results, we shall use it to create an algorithm to classify humans and dogs in the next jupyter notebook (IV).

### 0.Import libraries

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# We start by importing libraries
import time
import json
import copy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
from PIL import Image
from collections import OrderedDict

import torch
from torch import nn, optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

import os


In [3]:
!pip3 install torch torchvision


In [4]:
import PIL
print(PIL.PILLOW_VERSION)

4.1.1


In [5]:
!apt-get install


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


In [6]:
# Downloading the dog and human dataset
from IPython.display import clear_output
!wget -cq -O dogImages.zip https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!wget -cq -O lfw.tgz http://vis-www.cs.umass.edu/lfw/lfw.tgz
clear_output()  
print("Downloaded Successfully")

Downloaded Successfully


In [7]:
# Extractting the datasets
!unzip -n dogImages.zip
!tar -xvzf lfw.tgz
clear_output()
print("Extracted Successfully")

Extracted Successfully


In [8]:
# Downloading the pre-trained face detector
import os
if not os.path.isdir('haarcascades'):
    os.mkdir('haarcascades')
!wget -cq -O haarcascades/haarcascade_frontalface_alt.xml https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/project-dog-classification/haarcascades/haarcascade_frontalface_alt.xml
clear_output()
print("Downloaded Successfully")

Downloaded Successfully


### 1.Prepare images for classification

In [0]:
#The following code will help us when dealing with images to be used in different classification models#
#from PIL import Image
#import torchvision.transforms as transforms
#from torch.autograd import Variable

def process_image_to_tensor(image):

    # define transforms for the training data and testing data
    prediction_transforms = transforms.Compose([transforms.Resize(224),
                                          transforms.CenterCrop(224),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    
    img_pil = Image.open( image ).convert('RGB')
    img_tensor = prediction_transforms( img_pil )[:3,:,:].unsqueeze(0)
    
    return img_tensor


# helper function for un-normalizing an image  - from STYLE TRANSFER exercise
# and converting it from a Tensor image to a NumPy image for display
def image_convert(tensor):
    """ This is to display a tensor as an image. """
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

#### 1.1 Specify Directories

In [0]:
# Specify directories
data_dir = 'dogImages'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [0]:
import os
from torchvision import datasets

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
# Batch size
batch_size = 20
# For faster computation, setting num_workers
num_workers = 0

# Transforms for the training, validation, and testing sets
data_transforms = {
    'train'      : transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])]),

    'valid'      : transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])]),
    'test'       : transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
}

# Loading the datasets with ImageFolder
image_datasets = {
    'train'  : datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid'  : datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test'   : datasets.ImageFolder(test_dir, transform=data_transforms['test'])
}

# Using the image datasets and the trainforms to define dataloaders
loaders = {
    'train' : torch.utils.data.DataLoader(image_datasets['train'], batch_size = 32, shuffle=True, num_workers = num_workers),
    'valid' : torch.utils.data.DataLoader(image_datasets['valid'], batch_size = 16),
    'test'  : torch.utils.data.DataLoader(image_datasets['test'], batch_size = 16)}

In [12]:
#Find out the number of images for each category
import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("lfw/*/*"))
dog_files = np.array(glob("dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

There are 13233 total human images.
There are 8351 total dog images.


### 2.Importing VGG-16

In [13]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Is GPU available: ', 'Yes' if torch.cuda.is_available() else 'No')
# check if CUDA is available
use_cuda = torch.cuda.is_available()

Is GPU available:  Yes


In [0]:
#Now I try to do the same but with VGG16 instead
import torchvision.models as models
import torch.nn as nn

# Loading the pretrained model
model_transfer2 = models.vgg16(pretrained=True)

if use_cuda:
    model_transfer2 = model_transfer2.cuda()

#### 2.1 VGG-16 model architecture

In this part I shall modify the model architecture, I want to use the first layers as feature extractor and then replace the last one for classifying my model. 

In [0]:
# freeze parameters so we don't backprop through them
for param in model_transfer2.features.parameters():
    param.requires_grad = False

# replace the last fully connected layer with a Linnear layer with 133 out features (param_output_size)
num_features = model_transfer2.classifier[6].in_features
features = list(model_transfer2.classifier.children())[:-1]
features.extend([nn.Linear(num_features, 133)])
model_transfer2.classifier = nn.Sequential(*features)

if use_cuda:
    model_transfer2 = model_transfer2.cuda()

#### 2.2 Specify Loss Function and Optimizer

In [0]:
import torch.optim as optim
# Selecting the loss function and optimizer
criterion_transfer2 = nn.CrossEntropyLoss()

optimizer_transfer2 = optim.SGD(model_transfer2.classifier.parameters(), lr=0.01)

# Moving the model to the device
model_transfer2.to(device);

#### 2.3 Train and Validate the model

In [0]:
# this part helps building a robust training to deal with truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    print("start training for {} epochs ...".format(n_epochs))

    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    # exist save-file, load save file
    if os.path.exists(save_path):
        print("load previous saved model ...")
        model.load_state_dict(torch.load(save_path))
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
     
        # print training/validation statistics 
        print('\n-----------------------------------------------------------------------------\nEpoch: {} \nTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss has decreased from ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [18]:
loaders_transfer2 = loaders

# train the model
model_transfer2 = train(2, loaders_transfer2, model_transfer2, optimizer_transfer2, criterion_transfer2, use_cuda, 'model_transfer2.pt')

start training for 2 epochs ...
load previous saved model ...

-----------------------------------------------------------------------------
Epoch: 1 
Training Loss: 0.958612 	Validation Loss: 0.440452
Validation loss has decreased from (inf --> 0.440452).  Saving model ...

-----------------------------------------------------------------------------
Epoch: 2 
Training Loss: 0.955146 	Validation Loss: 0.394805
Validation loss has decreased from (0.440452 --> 0.394805).  Saving model ...


The model has been trained for 15 epochs through different rounds

#### 2.4 Test the model 

In [19]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders, model_transfer2, criterion_transfer2, use_cuda)

Test Loss: 0.467801


Test Accuracy: 85% (715/836)


### 2.5 Save the model


In [20]:
print("Our model: \n\n", model_transfer2, '\n')
print("The state dict keys: \n\n", model_transfer2.state_dict().keys())

Our model: 

 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  

In [0]:
checkpoint = {
    "classifier": model_transfer2.classifier,
    "state_dict": model_transfer2.state_dict()
}
torch.save(checkpoint, 'checkpoint.pth')

### 3.Evaluation 

We can see now accuracy of the classifier using VGG -16 goes up to 85% in only 11 epochs.  
Let's test on another document another top notch classifier, VGG-16

Certainly, I could improve the model by further adding elements to data agumentation, and by increasing the number of epochs. 

Comparing DENSENET 161 and VGG - 16, clearly we can see VGG-16 produces better results. 

So I will take VGG - 16 to write an algoirthm to detect humans, dogs or niether. This will be done in a new jupyter notebook. 
